In [ ]:
"""
Sequence to sequence
입력된 정보에서 추출된 정보들을 하나의 벡터로 만듬
encoder = 전체 입력되어지는 문장에대한 의미
encoder-> context벡터 출력 -> decoder ->기계번역
context벡터 = 실수형태의 값들이 저장되어있는 벡터 : 압축되어있는 정보
encdoer 와 decorder에서는 모두 RNN 구조를 사용(hidden_stats)

*Sequence to sequence ATTENTION모델
-모든 각각에 셀에 있는 hiddenstates가 다 넘어감
-encoder 쪽에서 decoder쪽으로 모든 스텝에 해당하는 hidden states가 넘어간다
-encoder에서 전달받은 전체 hidden states를 확인한다 -> 각각의 hidden state는 이전단어에 대한 입력단어에 대한 정보도 있고
현재 단어에 대한 입력 정보도 있는데 이러한 hidden state마다 점수를 계산한다 -> 계산된 점수에 대해 softmax를 취하고 
각각의 벡터하고 연산을 취함 -> 어느 단어에 집중해야할지 나옴

가장 아래쪽에 있는 encoder에서 임베딩 시작 -> 512차원에 대한 임베딩
-> 출력이 그다음 encoder로 이동

-self-attention기법을 통해 단어 간 연결시켜줌 -> 입력 문장 속에 다른 단어들을 보고 연관성을 파악해 현재 타겟위치의 단어를 인코딩함

-입력 단어(벡터)가 들어오면 세개의 벡터(학습가능한)가 만들어지고 이들을 Query 벡터, Key 벡터, Value 벡터라고 부름

score 계산 :현재위치에 있는 단어의 query vector와 다른 위치에 있는 단어의 key vector내적해 구함
->이후 현재 위치의 단어에 각각의 단어들이 어느정도로 점수를 갖느냐를 softmax를 통해 나타낸다
->softmax 값과 각각의 value값을 연산하는데 이때 없앨단어들은 value값을 낮게 주어 낮은 값을 갖도록해 없앤다


1.입력
2.임베딩
3.각각의 행렬과 가중치(Q,K,V)간의 연산 
4.Z라는 행렬 생성...
"""

In [36]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [37]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def download_zip(url, output_path):
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"ZIP file downloaded to {output_path}")
    else:
        print(f"Failed to download. HTTP Response Code: {response.status_code}")

url = "http://www.manythings.org/anki/fra-eng.zip"
output_path = "fra-eng.zip"
download_zip(url, output_path)

path = os.getcwd()
zipfilename = os.path.join(path, output_path)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

ZIP file downloaded to fra-eng.zip


In [38]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 227815


In [39]:
lines

,src,tar
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !
...,...,...
227810,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
227811,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...
227812,If someone who doesn't know your background sa...,Si quelqu'un qui ne connaît pas vos antécédent...
227813,It may be impossible to get a completely error...,Il est peut-être impossible d'obtenir un Corpu...


In [40]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

,src,tar
32640,He had an accident.,Il eut un accident.
8481,Is Tom afraid?,Tom a-t-il peur ?
37830,When were you born?,Quand êtes-vous nés ?
14925,He is a student.,C'est un étudiant.
10523,Did I break it?,L'ai-je cassé ?
57427,I want to get married.,Je veux me marier.
38405,You won't be fired.,Vous ne serez pas virées.
19671,Evil always wins.,Le mal gagne toujours.
3169,Here we are!,Nous voici !
49244,I was talking to you.,Je vous parlais.


In [41]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')   #\t가 시작, \n이 종료 심볼
lines.sample(10)

,src,tar
42220,It was unpardonable.,\t C'était inexcusable. \n
10635,Does that work?,\t Est-ce que cela fonctionne ? \n
51312,That's a big mistake.,\t C'est une grosse erreur. \n
2402,Let's kiss.,\t Embrassons-nous. \n
38157,You can trust them.,\t On peut leur faire confiance. \n
35491,Pets are permitted.,\t Les animaux sont autorisés. \n
38339,You party too much.,\t Vous faites trop la fête. \n
40256,I broke up with Tom.,\t Je me suis séparée de Tom. \n
17967,Tom never stops.,\t Tom ne s'arrête jamais. \n
40038,He's really selfish.,\t Il est vraiment égoïste. \n


In [42]:
# 문자 집합 구축
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

In [43]:
src_vocab

{' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ï',
 '’',
 '€'}

In [44]:
tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [45]:
tar_vocab

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'Ô',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f',
 '‽'}

In [46]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 80
target 문장의 char 집합 : 104


In [49]:
src_vocab=sorted(list(src_vocab))

In [50]:
src_vocab

[' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é',
 'ï',
 '’',
 '€']

In [51]:
tar_vocab = sorted(list(tar_vocab))

In [52]:
tar_vocab

['\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Ç',
 'É',
 'Ê',
 'Ô',
 'à',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ù',
 'û',
 'œ',
 '\u2009',
 '‘',
 '’',
 '\u202f',
 '‽']

In [53]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)]) # -> key자리에 value, value자리에 key로 바꿔줌 + 0 안씀
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, 'ï': 77, '’': 78, '€': 79}


In [54]:
print(tar_to_index)

{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 35, 'J': 36, 'K': 37, 'L': 38, 'M': 39, 'N': 40, 'O': 41, 'P': 42, 'Q': 43, 'R': 44, 'S': 45, 'T': 46, 'U': 47, 'V': 48, 'W': 49, 'X': 50, 'Y': 51, 'a': 52, 'b': 53, 'c': 54, 'd': 55, 'e': 56, 'f': 57, 'g': 58, 'h': 59, 'i': 60, 'j': 61, 'k': 62, 'l': 63, 'm': 64, 'n': 65, 'o': 66, 'p': 67, 'q': 68, 'r': 69, 's': 70, 't': 71, 'u': 72, 'v': 73, 'w': 74, 'x': 75, 'y': 76, 'z': 77, '\xa0': 78, '«': 79, '»': 80, 'À': 81, 'Ç': 82, 'É': 83, 'Ê': 84, 'Ô': 85, 'à': 86, 'â': 87, 'ç': 88, 'è': 89, 'é': 90, 'ê': 91, 'ë': 92, 'î': 93, 'ï': 94, 'ô': 95, 'ù': 96, 'û': 97, 'œ': 98, '\u2009': 99, '‘': 100, '’': 101, '\u202f': 102, '‽': 103}


In [60]:
encoder_input = []  #[30,64],[],....,[]


In [61]:
for line in lines.src: # 문장단위
    encoded_line=[]
    for c in line:   #문자단위
        encoded_line.append(src_to_index[c])
    encoder_input.append(encoded_line)

In [63]:
encoder_input #문자를 구성하는 숫자 단위로 나옴

[[30, 64, 10],
 [30, 64, 10],
 [30, 64, 10],
 [30, 64, 10],
 [31, 58, 10],
 [31, 58, 10],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 2],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [41, 70, 63, 10],
 [46, 57, 64, 23],
 [46, 64, 72, 2],
 [46, 64, 72, 2],
 [46, 64, 72, 2],
 [27, 70, 52, 60, 2],
 [27, 70, 52, 60, 2],
 [27, 70, 52, 60, 2],
 [29, 58, 67, 54, 2],
 [31, 54, 61, 65, 2],
 [31, 58, 53, 54, 10],
 [31, 58, 53, 54, 10],
 [33, 70, 62, 65, 2],
 [33, 70, 62, 65, 10],
 [42, 69, 64, 65, 2],
 [42, 69, 64, 65, 2],
 [42, 69, 64, 65, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 2],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [46, 50, 58, 69, 10],
 [25, 54, 56, 58, 63, 10],
 [25, 54, 56, 58, 63, 10],
 [30, 64, 1, 64, 63, 10],
 [30, 64, 1, 64, 63, 10],
 [30, 64, 

In [64]:
decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 48, 52, 3, 4, 3, 2], [1, 3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [1, 3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [1, 3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [1, 3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [65]:
# 모든 문장의 앞에있는 \t 제거함 
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 48, 52, 3, 4, 3, 2], [3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [66]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 76


In [67]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [68]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [74]:
encoder_input.shape #(60000, 22, 80)  ->60000건, 22는 최대길이
decoder_input.shape #(60000, 76, 104)
decoder_target.shape #(60000, 76, 104)

(60000, 76, 104)

In [75]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
#Seq2Seq 인코더 설계

In [76]:
encoder_inputs = Input(shape=(None, src_vocab_size))

In [79]:
#모든 rnn셀에서 전달되어지는 값을 가지고 마지막 셀에서 return_state=True를 설정하면 context vector로 출력됨
encoder_lstm=LSTM(units=256, return_state=True)

In [77]:
src_vocab_size #문자 벡터의 크기 

80

In [80]:
encoder_outputs, state_h, state_c =encoder_lstm(encoder_inputs)
#state_h(은닉 상태), state_c(셀 상태)

In [81]:
encoder_states = [state_h, state_c]  #encoder_states: 컨텍스트 벡터

In [ ]:
#Seq2Seq 디코더 설계

In [82]:
decoder_inputs = Input(shape=(None, tar_vocab_size))

In [84]:
decoder_lstm = LSTM(units=256, return_state=True, return_sequences=True)  #decoder에서는 결과값 나와야 함으로 sequences=True

In [88]:
decoder_outputs, _, _ =decoder_lstm(decoder_inputs, initial_state=encoder_states) #encoder의 states 값들이 decoder의 초기 값으로 옴
#decoder은 은닉 상태, 셀 상태 정보가 필요 없음(훈련과정에서는)

In [86]:
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')

In [89]:
decoder_outputs=decoder_softmax_layer(decoder_outputs) 
# decoder_outputs는 256차원, decoder_softmax_layer은 104차원임으로 맞춰줌

In [90]:
model=Model([encoder_inputs, decoder_inputs],decoder_outputs) #[입력],츌력

In [92]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [93]:
model.fit(x=[encoder_input,decoder_input], y=decoder_target,batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
750/750 [==============================] - 124s 162ms/step - loss: 0.8535 - val_loss: 0.7626
Epoch 2/40
750/750 [==============================] - 129s 172ms/step - loss: 0.5710 - val_loss: 0.6614
Epoch 3/40
750/750 [==============================] - 124s 165ms/step - loss: 0.5024 - val_loss: 0.6015
Epoch 4/40
750/750 [==============================] - 121s 161ms/step - loss: 0.4567 - val_loss: 0.5599
Epoch 5/40
750/750 [==============================] - 123s 164ms/step - loss: 0.4236 - val_loss: 0.5266
Epoch 6/40
750/750 [==============================] - 124s 166ms/step - loss: 0.3969 - val_loss: 0.4910
Epoch 7/40
750/750 [==============================] - 120s 161ms/step - loss: 0.3758 - val_loss: 0.5235
Epoch 8/40
750/750 [==============================] - 120s 160ms/step - loss: 0.3592 - val_loss: 0.4552
Epoch 9/40
750/750 [==============================] - 121s 161ms/step - loss: 0.3453 - val_loss: 0.4410
Epoch 10/40
750/750 [==============================] - 120s 160m

In [ ]:
#테스트 과정
1. 번역 대상 입력 문장이 인코더에 들어감 ->은닉/셀 상태 ->디코더
2. 디코더의 입력 시그널이 전ㅠ달(sos,\t)
3. 디코더는 다음 문자 예측

In [95]:
#테스트 과정에서의 디코더 정의

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c] #hidden state와 cell state 두개씩 전달 

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [96]:
#테스트 과정에서의 인코더 정의
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [135]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [142]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq,verbose=0)
  # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ""
  # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value,verbose=0)
    # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
    # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char
    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
            len(decoded_sentence) > max_tar_len):
            stop_condition = True
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.
        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]
    return decoded_sentence

In [143]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Dégage ! 
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Salut ! 
-----------------------------------
입력 문장: Got it!
정답 문장: J'ai pigé ! 
번역 문장: Tomme ! 
-----------------------------------
입력 문장: Go home.
정답 문장: Rentre à la maison. 
번역 문장: Va au lit ! 
-----------------------------------
입력 문장: Get going.
정답 문장: En avant. 
번역 문장: Dégage ! 
